# Multiplication_rnn

In [1]:
from __future__ import print_function
from keras.models import Sequential
from keras import layers
import numpy as np
from six.moves import range

Using TensorFlow backend.


## Data Representation

In [2]:
class CharacterTable(object):
    """Given a set of characters:
    + Encode them to a one-hot integer representation
    + Decode the one-hot or integer representation to their character output
    + Decode a vector of probabilities to their character output
    """
    def __init__(self, chars):
        """Initialize character table.
        # Arguments
            chars: Characters that can appear in the input.
        """
        self.chars = sorted(set(chars))
        self.char_indices = dict((c, i) for i, c in enumerate(self.chars))
        self.indices_char = dict((i, c) for i, c in enumerate(self.chars))

    def encode(self, C, num_rows):
        """One-hot encode given string C.
        # Arguments
            C: string, to be encoded.
            num_rows: Number of rows in the returned one-hot encoding. This is
                used to keep the # of rows for each data the same.
        """
        x = np.zeros((num_rows, len(self.chars)))
        for i, c in enumerate(C):
            x[i, self.char_indices[c]] = 1
        return x

    def decode(self, x, calc_argmax=True):
        """Decode the given vector or 2D array to their character output.
        # Arguments
            x: A vector or a 2D array of probabilities or one-hot representations;
                or a vector of character indices (used with `calc_argmax=False`).
            calc_argmax: Whether to find the character index with maximum
                probability, defaults to `True`.
        """
        if calc_argmax:
            x = x.argmax(axis=-1)
        return ''.join(self.indices_char[x] for x in x)


In [3]:
class colors:
    ok = '\033[92m'
    fail = '\033[91m'
    close = '\033[0m'

In [4]:
# Parameters for the model and dataset.
TRAINING_SIZE = 50000
DIGITS = 3
REVERSE = True

# Maximum length of input is 'int + int' (e.g., '345+678'). Maximum length of
# int is DIGITS.
MAXLEN = DIGITS + 1 + DIGITS

# All the numbers, plus sign and space for padding.
chars = '0123456789* '     #####
ctable = CharacterTable(chars)

## Data Generation

In [5]:
def Generate_Data(DIGITS, DATA_SIZE, MAXLEN):
    questions = []
    expected = []
    seen = set()
    print('Generating data...')
    while len(questions) < DATA_SIZE:    ## the amount of generated data is TRAINING_SIZE
        f = lambda: int(''.join(np.random.choice(list('0123456789'))
                        for i in range(np.random.randint(1, DIGITS + 1))))
        a, b = f(), f()


        # Skip any addition questions we've already seen
        # Also skip any such that x+Y == Y+x (hence the sorting).
        key = tuple(sorted((a, b)))
        if key in seen:
            continue
        seen.add(key)
        # Pad the data with spaces such that it is always MAXLEN.
        q = '{}*{}'.format(a, b)    #####
        query = q + ' ' * (MAXLEN - len(q))
        ans = str(a * b)   #####
        # Answers can be of maximum size DIGITS + 1.
        ans += ' ' * (DIGITS + 3 - len(ans))
        if REVERSE:
            # Reverse the query, e.g., '12+345  ' becomes '  543+21'. (Note the
            # space used for padding.)
            query = query[::-1]
        questions.append(query)
        expected.append(ans)

    print('Total addition questions:', len(questions))
    return questions, expected

## Feature Engineering

In [6]:
def Vectorization(questions, expected):
    print('Vectorization...')
    x = np.zeros((len(questions), MAXLEN, len(chars)), dtype=np.bool)
    y = np.zeros((len(questions), DIGITS + 3, len(chars)), dtype=np.bool)
    for i, sentence in enumerate(questions):
        x[i] = ctable.encode(sentence, MAXLEN)
    for i, sentence in enumerate(expected):
        y[i] = ctable.encode(sentence, DIGITS + 3)

    return x, y

## Get Training Data and Validation Data
* total data = 50000, 45000 for training, 5000 for validation

In [7]:
questions, expected = Generate_Data(DIGITS = DIGITS, DATA_SIZE = TRAINING_SIZE, MAXLEN = MAXLEN)
x, y = Vectorization(questions, expected)

# Shuffle (x, y) in unison as the later parts of x will almost all be larger
# digits.
indices = np.arange(len(y))
np.random.shuffle(indices)
x = x[indices]
y = y[indices]

# Explicitly set apart 10% for validation data that we never train over.
split_at = len(x) - len(x) // 10
(x_train, x_val) = x[:split_at], x[split_at:]
(y_train, y_val) = y[:split_at], y[split_at:]

print('Training Data:')
print(x_train.shape)
print(y_train.shape)

print('Validation Data:')
print(x_val.shape)
print(y_val.shape)

Generating data...
Total addition questions: 50000
Vectorization...
Training Data:
(45000, 7, 12)
(45000, 6, 12)
Validation Data:
(5000, 7, 12)
(5000, 6, 12)


## Build Model

In [8]:
def Build_Model(RNN, HIDDEN_SIZE, BATCH_SIZE, LAYERS, MAXLEN, DIGITS, chars):
    print('Build model...')
    model = Sequential()
    
    model.add(RNN(HIDDEN_SIZE, input_shape=(MAXLEN, len(chars))))
    model.add(layers.Dense((DIGITS + 3) * 512))
    model.add(layers.Reshape(((DIGITS + 3), 512)))
    model.add(RNN(HIDDEN_SIZE, return_sequences=True))
    model.add(layers.TimeDistributed(layers.Dense(len(chars), activation='softmax')))

    model.compile(loss='categorical_crossentropy',
                  optimizer='adam',
                  metrics=['accuracy'])
    model.summary()
    
    return model

In [9]:
def Train_Step(model, epoch, REVERSE = True):
    for iteration in range(1, epoch):
        print()
        print('-' * 50)
        print('Iteration', iteration)
        model.fit(x_train, y_train,
                  batch_size=BATCH_SIZE,
                  epochs=1,
                  validation_data=(x_val, y_val))
        # Select 10 samples from the validation set at random so we can visualize
        # errors.
        for i in range(10):
            ind = np.random.randint(0, len(x_val))
            rowx, rowy = x_val[np.array([ind])], y_val[np.array([ind])]
            preds = model.predict_classes(rowx, verbose=0)
            q = ctable.decode(rowx[0])
            correct = ctable.decode(rowy[0])
            guess = ctable.decode(preds[0], calc_argmax=False)
            print('Q', q[::-1] if REVERSE else q, end=' ')
            print('T', correct, end=' ')
            if correct == guess:
                print(colors.ok + '☑' + colors.close, end=' ')
            else:
                print(colors.fail + '☒' + colors.close, end=' ')
            print(guess)

## String Matching
* training 300 epochs

In [10]:
RNN = layers.LSTM
HIDDEN_SIZE = 512   #128
BATCH_SIZE = 128
LAYERS = 1

model = Build_Model(RNN = RNN, HIDDEN_SIZE = HIDDEN_SIZE, BATCH_SIZE = BATCH_SIZE, LAYERS = LAYERS, MAXLEN = MAXLEN, DIGITS = DIGITS, chars = chars)

Build model...
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, 512)               1075200   
_________________________________________________________________
dense_1 (Dense)              (None, 3072)              1575936   
_________________________________________________________________
reshape_1 (Reshape)          (None, 6, 512)            0         
_________________________________________________________________
lstm_2 (LSTM)                (None, 6, 512)            2099200   
_________________________________________________________________
time_distributed_1 (TimeDist (None, 6, 12)             6156      
Total params: 4,756,492
Trainable params: 4,756,492
Non-trainable params: 0
_________________________________________________________________


In [11]:
Train_Step(model, epoch = 200)


--------------------------------------------------
Iteration 1
Train on 45000 samples, validate on 5000 samples
Epoch 1/1
45000/45000 [==============================] - 15s 335us/step - loss: 1.7536 - acc: 0.3457 - val_loss: 1.5917 - val_acc: 0.3929
Q 79*704  T 55616  ☒ 55920 
Q 60*723  T 43380  ☒ 29920 
Q 163*140 T 22820  ☒ 62800 
Q 204*28  T 5712   ☒ 1288  
Q 295*87  T 25665  ☒ 22955 
Q 33*453  T 14949  ☒ 22255 
Q 394*235 T 92590  ☒ 12225 
Q 492*525 T 258300 ☒ 222250
Q 68*39   T 2652   ☒ 2994  
Q 690*48  T 33120  ☒ 39980 

--------------------------------------------------
Iteration 2
Train on 45000 samples, validate on 5000 samples
Epoch 1/1
45000/45000 [==============================] - 12s 267us/step - loss: 1.3427 - acc: 0.4677 - val_loss: 1.2337 - val_acc: 0.4976
Q 581*380 T 220780 ☒ 208380
Q 997*6   T 5982   ☒ 6958  
Q 50*522  T 26100  ☒ 24100 
Q 498*20  T 9960   ☒ 9880  
Q 2*29    T 58     ☒ 68    
Q 4*938   T 3752   ☒ 3788  
Q 181*3   T 543    ☒ 519   
Q 97*28   T 2716   ☒ 2

45000/45000 [==============================] - 12s 262us/step - loss: 0.6201 - acc: 0.7560 - val_loss: 0.6662 - val_acc: 0.7330
Q 8*282   T 2256   ☒ 2276  
Q 33*521  T 17193  ☒ 17353 
Q 461*87  T 40107  ☒ 49087 
Q 757*431 T 326267 ☒ 329887
Q 63*79   T 4977   ☒ 4897  
Q 772*9   T 6948   ☒ 6928  
Q 508*53  T 26924  ☒ 26704 
Q 96*19   T 1824   ☒ 1764  
Q 73*350  T 25550  ☒ 25450 
Q 8*547   T 4376   ☒ 4496  

--------------------------------------------------
Iteration 15
Train on 45000 samples, validate on 5000 samples
Epoch 1/1
45000/45000 [==============================] - 12s 262us/step - loss: 0.6027 - acc: 0.7643 - val_loss: 0.6452 - val_acc: 0.7403
Q 7*439   T 3073   ☒ 3113  
Q 7*611   T 4277   ☒ 4377  
Q 8*429   T 3432   ☒ 3412  
Q 8*372   T 2976   ☑ 2976  
Q 838*971 T 813698 ☒ 811708
Q 394*235 T 92590  ☒ 95490 
Q 4*331   T 1324   ☑ 1324  
Q 89*600  T 53400  ☑ 53400 
Q 413*49  T 20237  ☒ 20737 
Q 92*205  T 18860  ☑ 18860 

--------------------------------------------------
Iteratio

Q 39*240  T 9360   ☒ 9460  
Q 818*253 T 206954 ☒ 202414
Q 90*312  T 28080  ☑ 28080 
Q 813*61  T 49593  ☑ 49593 
Q 328*848 T 278144 ☒ 272724
Q 41*303  T 12423  ☑ 12423 
Q 49*162  T 7938   ☒ 8198  
Q 523*793 T 414739 ☒ 410179
Q 255*51  T 13005  ☒ 13605 
Q 247*491 T 121277 ☒ 129137

--------------------------------------------------
Iteration 28
Train on 45000 samples, validate on 5000 samples
Epoch 1/1
45000/45000 [==============================] - 12s 263us/step - loss: 0.4370 - acc: 0.8388 - val_loss: 0.6030 - val_acc: 0.7727
Q 0*769   T 0      ☑ 0     
Q 229*76  T 17404  ☒ 17024 
Q 654*5   T 3270   ☑ 3270  
Q 4*667   T 2668   ☑ 2668  
Q 15*613  T 9195   ☑ 9195  
Q 114*379 T 43206  ☒ 43826 
Q 75*92   T 6900   ☒ 6800  
Q 68*715  T 48620  ☒ 48320 
Q 87*300  T 26100  ☑ 26100 
Q 200*11  T 2200   ☑ 2200  

--------------------------------------------------
Iteration 29
Train on 45000 samples, validate on 5000 samples
Epoch 1/1
45000/45000 [==============================] - 12s 262us/step - 

45000/45000 [==============================] - 12s 262us/step - loss: 0.2758 - acc: 0.9091 - val_loss: 0.6691 - val_acc: 0.7803
Q 60*387  T 23220  ☑ 23220 
Q 316*522 T 164952 ☒ 161752
Q 936*5   T 4680   ☑ 4680  
Q 82*314  T 25748  ☒ 25948 
Q 27*753  T 20331  ☒ 20231 
Q 307*15  T 4605   ☑ 4605  
Q 844*351 T 296244 ☒ 299644
Q 76*542  T 41192  ☒ 41052 
Q 26*644  T 16744  ☒ 16424 
Q 273*987 T 269451 ☒ 262171

--------------------------------------------------
Iteration 42
Train on 45000 samples, validate on 5000 samples
Epoch 1/1
45000/45000 [==============================] - 12s 263us/step - loss: 0.2722 - acc: 0.9092 - val_loss: 0.6796 - val_acc: 0.7805
Q 31*487  T 15097  ☒ 15317 
Q 54*332  T 17928  ☒ 17148 
Q 919*65  T 59735  ☒ 59935 
Q 388*59  T 22892  ☒ 24492 
Q 56*148  T 8288   ☒ 8028  
Q 294*192 T 56448  ☒ 57448 
Q 76*549  T 41724  ☒ 41524 
Q 793*775 T 614575 ☒ 614025
Q 252*496 T 124992 ☒ 125592
Q 69*38   T 2622   ☑ 2622  

--------------------------------------------------
Iteratio

Q 527*94  T 49538  ☒ 49118 
Q 252*348 T 87696  ☒ 86716 
Q 383*5   T 1915   ☒ 1815  
Q 73*971  T 70883  ☒ 71363 
Q 2*687   T 1374   ☑ 1374  
Q 788*865 T 681620 ☒ 682620
Q 187*2   T 374    ☑ 374   
Q 639*32  T 20448  ☒ 20908 
Q 637*520 T 331240 ☒ 332840
Q 196*846 T 165816 ☒ 162416

--------------------------------------------------
Iteration 55
Train on 45000 samples, validate on 5000 samples
Epoch 1/1
45000/45000 [==============================] - 12s 263us/step - loss: 0.1359 - acc: 0.9612 - val_loss: 0.8487 - val_acc: 0.7789
Q 256*3   T 768    ☒ 778   
Q 226*59  T 13334  ☒ 13074 
Q 25*99   T 2475   ☑ 2475  
Q 24*564  T 13536  ☒ 13176 
Q 824*8   T 6592   ☑ 6592  
Q 159*267 T 42453  ☒ 47373 
Q 78*733  T 57174  ☒ 56374 
Q 321*987 T 316827 ☒ 317167
Q 918*53  T 48654  ☒ 48124 
Q 620*257 T 159340 ☒ 161740

--------------------------------------------------
Iteration 56
Train on 45000 samples, validate on 5000 samples
Epoch 1/1
45000/45000 [==============================] - 12s 263us/step - 

...

Q 789*872 T 688008 ☒ 796318
Q 31*421  T 13051  ☒ 12751 
Q 750*442 T 331500 ☒ 333500
Q 361*838 T 302518 ☒ 308778
Q 30*173  T 5190   ☒ 5790  
Q 14*364  T 5096   ☒ 5076  
Q 398*993 T 395214 ☒ 392014
Q 263*371 T 97573  ☒ 100773
Q 728*421 T 306488 ☒ 301988
Q 25*106  T 2650   ☒ 2700  

--------------------------------------------------
Iteration 136
Train on 45000 samples, validate on 5000 samples
Epoch 1/1
45000/45000 [==============================] - 12s 263us/step - loss: 0.0179 - acc: 0.9963 - val_loss: 1.2904 - val_acc: 0.7824
Q 830*264 T 219120 ☒ 217720
Q 11*507  T 5577   ☒ 5557  
Q 73*746  T 54458  ☒ 54678 
Q 461*39  T 17979  ☒ 17699 
Q 905*495 T 447975 ☒ 449975
Q 978*96  T 93888  ☒ 94668 
Q 623*33  T 20559  ☒ 20899 
Q 537*302 T 162174 ☒ 165154
Q 735*11  T 8085   ☒ 8025  
Q 248*92  T 22816  ☒ 23216 

--------------------------------------------------
Iteration 137
Train on 45000 samples, validate on 5000 samples
Epoch 1/1
45000/45000 [==============================] - 12s 262us/step 

45000/45000 [==============================] - 12s 263us/step - loss: 0.0072 - acc: 0.9990 - val_loss: 1.3293 - val_acc: 0.7849
Q 636*389 T 247404 ☒ 248424
Q 8*447   T 3576   ☒ 3536  
Q 416*288 T 119808 ☒ 112188
Q 63*526  T 33138  ☒ 33938 
Q 192*584 T 112128 ☒ 112908
Q 910*15  T 13650  ☑ 13650 
Q 72*622  T 44784  ☑ 44784 
Q 431*943 T 406433 ☒ 401113
Q 307*41  T 12587  ☒ 12747 
Q 697*699 T 487203 ☒ 491703

--------------------------------------------------
Iteration 150
Train on 45000 samples, validate on 5000 samples
Epoch 1/1
45000/45000 [==============================] - 12s 263us/step - loss: 0.0170 - acc: 0.9959 - val_loss: 1.3781 - val_acc: 0.7778
Q 206*59  T 12154  ☒ 12054 
Q 993*47  T 46671  ☒ 46271 
Q 18*383  T 6894   ☒ 6214  
Q 58*356  T 20648  ☒ 20848 
Q 679*93  T 63147  ☒ 63567 
Q 15*168  T 2520   ☒ 2420  
Q 12*545  T 6540   ☑ 6540  
Q 53*386  T 20458  ☒ 20488 
Q 52*276  T 14352  ☒ 14952 
Q 44*727  T 31988  ☒ 31088 

--------------------------------------------------
Iterati

45000/45000 [==============================] - 12s 264us/step - loss: 0.0088 - acc: 0.9982 - val_loss: 1.3927 - val_acc: 0.7824
Q 50*52   T 2600   ☑ 2600  
Q 835*948 T 791580 ☒ 795280
Q 531*344 T 182664 ☒ 181964
Q 90*908  T 81720  ☑ 81720 
Q 37*479  T 17723  ☒ 17803 
Q 6*514   T 3084   ☑ 3084  
Q 123*241 T 29643  ☒ 29613 
Q 848*68  T 57664  ☒ 57084 
Q 941*118 T 111038 ☒ 102558
Q 312*78  T 24336  ☒ 24616 

--------------------------------------------------
Iteration 163
Train on 45000 samples, validate on 5000 samples
Epoch 1/1
45000/45000 [==============================] - 12s 263us/step - loss: 0.0246 - acc: 0.9934 - val_loss: 1.4579 - val_acc: 0.7686
Q 70*847  T 59290  ☒ 59490 
Q 573*8   T 4584   ☒ 4664  
Q 177*90  T 15930  ☒ 16130 
Q 59*27   T 1593   ☒ 1693  
Q 52*576  T 29952  ☒ 29172 
Q 922*96  T 88512  ☒ 88572 
Q 54*790  T 42660  ☑ 42660 
Q 123*78  T 9594   ☒ 9434  
Q 296*385 T 113960 ☒ 114360
Q 871*28  T 24388  ☒ 24008 

--------------------------------------------------
Iterati

45000/45000 [==============================] - 12s 263us/step - loss: 0.0041 - acc: 0.9993 - val_loss: 1.4095 - val_acc: 0.7857
Q 48*160  T 7680   ☑ 7680  
Q 350*691 T 241850 ☑ 241850
Q 247*342 T 84474  ☒ 82874 
Q 372*936 T 348192 ☒ 353292
Q 708*948 T 671184 ☒ 666184
Q 305*882 T 269010 ☒ 272110
Q 735*109 T 80115  ☒ 80915 
Q 738*36  T 26568  ☒ 26768 
Q 915*39  T 35685  ☒ 35385 
Q 922*9   T 8298   ☒ 8398  

--------------------------------------------------
Iteration 176
Train on 45000 samples, validate on 5000 samples
Epoch 1/1
45000/45000 [==============================] - 12s 262us/step - loss: 0.0566 - acc: 0.9831 - val_loss: 1.4891 - val_acc: 0.7664
Q 247*209 T 51623  ☒ 52783 
Q 669*5   T 3345   ☑ 3345  
Q 169*969 T 163761 ☒ 176981
Q 395*96  T 37920  ☒ 37120 
Q 726*20  T 14520  ☑ 14520 
Q 735*529 T 388815 ☒ 386415
Q 868*70  T 60760  ☑ 60760 
Q 952*832 T 792064 ☒ 799644
Q 0*359   T 0      ☒ 05    
Q 984*181 T 178104 ☒ 178084

--------------------------------------------------
Iterati

Q 59*327  T 19293  ☒ 19473 
Q 724*97  T 70228  ☒ 60248 
Q 49*542  T 26558  ☒ 26118 
Q 60*838  T 50280  ☑ 50280 
Q 135*928 T 125280 ☒ 132280
Q 394*8   T 3152   ☒ 3132  
Q 107*88  T 9416   ☒ 9336  
Q 243*392 T 95256  ☒ 99716 
Q 763*9   T 6867   ☑ 6867  
Q 49*269  T 13181  ☒ 13281 

--------------------------------------------------
Iteration 189
Train on 45000 samples, validate on 5000 samples
Epoch 1/1
45000/45000 [==============================] - 12s 263us/step - loss: 7.1801e-04 - acc: 1.0000 - val_loss: 1.4218 - val_acc: 0.7891
Q 41*988  T 40508  ☒ 40448 
Q 44*403  T 17732  ☒ 17752 
Q 56*38   T 2128   ☒ 2188  
Q 937*980 T 918260 ☒ 916060
Q 11*389  T 4279   ☒ 4379  
Q 177*75  T 13275  ☑ 13275 
Q 229*600 T 137400 ☒ 133400
Q 181*3   T 543    ☑ 543   
Q 316*929 T 293564 ☒ 298184
Q 20*151  T 3020   ☑ 3020  

--------------------------------------------------
Iteration 190
Train on 45000 samples, validate on 5000 samples
Epoch 1/1
45000/45000 [==============================] - 12s 262us/s

## Testing

In [12]:
## Generate Testing Data
questions_test, expected_test = Generate_Data(DIGITS = DIGITS, DATA_SIZE = 1000, MAXLEN = MAXLEN)
x_test, y_test = Vectorization(questions_test, expected_test)

Generating data...
Total addition questions: 1000
Vectorization...


In [13]:
## Testing
count_correct = 0
preds = model.predict_classes(x_test, verbose=0)
print("Visualize 10 Data (Total Testing Data = 1000)")
for i in range(1000):
    q = ctable.decode(x_test[i])
    correct = ctable.decode(y_test[i])
    guess = ctable.decode(preds[i], calc_argmax=False)
    if correct == guess:
        count_correct += 1
    if i < 10:
        print('Q', q[::-1] if REVERSE else q, end=' ')
        print('T', correct, end=' ')
        if correct == guess:
            print(colors.ok + '☑' + colors.close, end=' ')
        else:
            print(colors.fail + '☒' + colors.close, end=' ')
        print(guess)
    
print("Testing Accuracy : ",(float)(count_correct / len(x_test)))

Visualize 10 Data (Total Testing Data = 1000)
Q 556*9   T 5004   ☑ 5004  
Q 72*75   T 5400   ☑ 5400  
Q 45*29   T 1305   ☑ 1305  
Q 7*3     T 21     ☑ 21    
Q 20*0    T 0      ☑ 0     
Q 1*228   T 228    ☑ 228   
Q 2*103   T 206    ☑ 206   
Q 515*79  T 40685  ☑ 40685 
Q 144*509 T 73296  ☒ 73256 
Q 31*20   T 620    ☑ 620   
Testing Accuracy :  0.684


## Result
### * 利用LSTM做encoder-decoder的seq2seq架構來實現乘法器
### * Training data總數為50000筆，45000為training，5000為validation
### * Batch size = 128
### * RNN hidden layer = 512
### * Optimizer使用adam
### * Epochs = 200
### * Validation result達78.99%
### * Testing data總數為1000筆，Testing accuracy = 68.4%